## 🔬 Project Overview: Clustering & Annotating PBMCs via Single-Cell RNA-Seq

This notebook demonstrates a complete and reproducible single-cell transcriptomics pipeline using peripheral blood mononuclear cell (PBMC) data. The goal is to clean the dataset, reduce its complexity, identify distinct cell types via clustering, and assign biological meaning using gene markers.

The workflow is designed to be understandable even for users with limited biology or bioinformatics background.

---

### 🔁 Step-by-Step Pipeline Summary

**1. 📥 Data Upload**  
I load a publicly available PBMC dataset as an `AnnData` object. Each row is a single cell, and each column is a gene.  
✅ No extra installation or data wrangling required.

---

**2. 🧹 Quality Control (QC)**  
I apply basic QC filters to remove low-quality or dying cells:  
- Cells with <200 genes are discarded.  
- Cells with >5% mitochondrial gene content are excluded. These are likely damaged.  
- Genes that are rarely expressed (<3 cells) are dropped.

**Why Mitochondrial Genes?**  
Dying cells often leak mitochondrial RNA. I flag these (`MT-` prefix) to filter out unhealthy cells before analysis.

---

**3. 🔧 Normalization & Feature Selection**  
- Normalize counts to make cells comparable.  
- Apply log transformation for stability.  
- Select 2,000 most variable genes (HVGs) to focus on informative features only.

---

**4. 📉 Dimensionality Reduction & UMAP Visualization**  
- Apply PCA to reduce dimensions.  
- Construct a neighborhood graph of cells.  
- Embed data in 2D using UMAP for visualization.  
- Overlay known marker genes (*e.g., CST3, NKG7, PPBP*) to identify broad immune cell types.

---

**5. 🧱 Unsupervised Clustering**  
- Use Leiden clustering to group cells based on transcriptome similarity.  
- Visualize clusters in UMAP space.

---

**6. 📈 Identify Cluster-Specific Marker Genes**  
- Use t-tests to find genes highly expressed in each cluster.  
- Visualize top 20 genes using:
  - Bar-style ranking plots  
  - Dotplot showing average expression and % of expressing cells

---

**7. 🧠 Manual Cell Type Annotation**  
I assign biological meaning to clusters based on known gene markers. For example:

| Cluster | Marker Genes         | Likely Cell Type      |
|---------|----------------------|------------------------|
| 0       | LTB, IL32, CD2       | T cells               |
| 1       | CST3, TYROBP, LST1   | Monocytes             |
| 2       | NKG7, GZMA, GNLY     | NK cells              |
| 3       | MS4A1, CD79A, HLA-DQ | B cells               |
| 4       | HLA-DPB1, HLA-DRB1   | Dendritic cells       |
| 5       | PPBP, PF4            | Platelets             |

Use known marker resources (e.g., PanglaoDB, Human Cell Atlas) to support annotation.  
This step **is manual** to ensure clarity and flexibility.

---

**8. 🎯 UMAP Overlays of Marker Genes**  
- Visualize expression of key markers across clusters.  
- Validate cell type identity with biological context.

---

### 📊 Expected Outputs

- Violin plots for QC inspection  
- UMAP with cluster IDs  
- Marker gene ranking per cluster  
- Dotplot comparing top genes across clusters  
- UMAPs for key immune marker genes

---

### 💡 Reproducibility Notes

- No private or local data needed  
- Uses open tools: Scanpy, AnnData, UMAP  
- All steps can be run on most laptops in minutes  
- Can be extended to other tissues or custom datasets

---





In [ ]:
# 1. Data upload

import scanpy as sc
import anndata
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

adata = sc.datasets.pbmc3k()
adata


In [ ]:
# 2. Quality control

adata.var["mt"] = adata.var_names.str.startswith("MT-")

sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)

sc.pp.filter_cells(adata, min_genes=200) # remove low-complexity cells
adata = adata[adata.obs.pct_counts_mt < 5, :] # remove cells likely to be stressed/dying (mito RNA dominates)
sc.pp.filter_genes(adata, min_cells=3) # remove genes expressed in fewer than 3 cells (too rare to analyze)

sc.pl.violin(adata, ["n_genes_by_counts", "total_counts", "pct_counts_mt"], jitter=0.4, multi_panel=True)

In [ ]:
# 3. Normalization & feature selection

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

adata = adata[:, adata.var.highly_variable]

In [ ]:
# 4. Dimentionality reduction & UMAP visualisation

sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver="arpack")
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)

sc.pl.umap(adata, color=["CST3", "NKG7", "PPBP"])

In [ ]:
# 5. Unsupervised clustering

sc.tl.leiden(adata, resolution=0.5)
sc.pl.umap(adata, color=["leiden"])

In [ ]:
# 6. Identify cluster-specific marker genes

sc.tl.rank_genes_groups(adata, groupby="leiden", method="t-test")
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, show=True)

sc.tl.dendrogram(adata, groupby="leiden")
sc.pl.rank_genes_groups_dotplot(adata, n_genes=5)

In [ ]:
# 7. Manual annotation based on Leiden cluster IDs

cluster2annotation = {
    '0': 'Naive T cells',
    '1': 'Monocytes',
    '2': 'NK cells',
    '3': 'B cells',
    '4': 'Dendritic cells',
    '5': 'Platelets'
}

# 8. UMAP overlays of marker genes

adata.obs['cell_type'] = adata.obs['leiden'].map(cluster2annotation)

fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(
    adata, 
    color='cell_type', 
    ax=ax,
    show=False,
    legend_loc='right margin',
    legend_fontsize='small',
    title='PBMC Cell Types (UMAP)'
)
plt.show()
